## Подключение диска и установка keras.tunes

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.0 MB/s eta 0:00:00


# Инициализация

In [3]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Dropout, LSTM, Bidirectional, SpatialDropout1D, Flatten, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from kerastuner.tuners import BayesianOptimization
from tensorflow import keras
from keras_tuner import Objective
import pandas as pd

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


<ipython-input-3-888032ba30a2>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization


In [4]:
# Максимальное количество слов
num_words = 10000
# Максимальная длина комментария
max_comment_len = 50
#Число классов
num_classes = 6

# Загрузка тренировочных данных

In [5]:
train = pd.read_csv('drive/MyDrive/toxicCommentsDataSet/toxicCommentsTrain.csv')

In [6]:
#Получение комментариев
comments = train['comment_text']

In [7]:
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# Токенизация

In [8]:
tokenizer = Tokenizer(num_words=num_words)

In [9]:
#Обучение токенайзера
tokenizer.fit_on_texts(comments)

In [10]:
train_sequences = tokenizer.texts_to_sequences(comments)

In [11]:
x_train = pad_sequences(train_sequences, maxlen=max_comment_len)

# Оптимизация гиперпараметров

In [12]:
def build_model(hp):
    activation_choice = hp.Choice('activation', values=['sigmoid', 'tanh', 'relu', 'elu', 'selu'])
    optimizer_choice = hp.Choice('optimizer', values=['SGD', 'adam', 'adagrad', 'adadelta', 'rmsprop'])
    layers_gru_num = hp.Int('layers_gru_num', 0, 2)
    neurons_num = hp.Int('neurons_num', min_value=4, max_value=50, step=8)
    embedding_num = hp.Int('embedding_num', min_value=4, max_value=128, step=16)
    embeddings_regularizer = keras.regularizers.l2(
        hp.Choice('embeddings_regularizer', values=[1e-2, 1e-3, 1e-4], default=1e-3))
    kernel_regularizer = keras.regularizers.l2(hp.Choice('kernel_regularizer', values=[1e-2, 1e-3, 1e-4], default=1e-3))

    model = Sequential()
    model.add(Embedding(input_dim=num_words, output_dim=embedding_num, input_length=max_comment_len,
                        embeddings_regularizer=embeddings_regularizer))
    for i in range(layers_gru_num):
        model.add(GRU(units=neurons_num, activation=activation_choice, kernel_regularizer=kernel_regularizer,return_sequences=True))
    model.add(GRU(units=neurons_num, activation=activation_choice, kernel_regularizer=kernel_regularizer))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=optimizer_choice, loss='binary_crossentropy', metrics=[keras.metrics.AUC(name="auc"), "accuracy"])

    return model

In [13]:
tuner = BayesianOptimization(
    build_model,
    objective=Objective('val_auc', 'max'),
    max_trials=50,
    directory='gru_toxic_dir'
)

In [14]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
activation (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'tanh', 'relu', 'elu', 'selu'], 'ordered': False}
optimizer (Choice)
{'default': 'SGD', 'conditions': [], 'values': ['SGD', 'adam', 'adagrad', 'adadelta', 'rmsprop'], 'ordered': False}
layers_gru_num (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 2, 'step': 1, 'sampling': 'linear'}
neurons_num (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 50, 'step': 8, 'sampling': 'linear'}
embedding_num (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
embeddings_regularizer (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
kernel_regularizer (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [15]:
tuner.search(x_train, y_train, batch_size=1024, epochs=5, validation_split=0.2)

Trial 50 Complete [00h 03m 26s]
val_auc: 0.9573037028312683

Best val_auc So Far: 0.9662976861000061
Total elapsed time: 02h 18m 47s


In [16]:
tuner.results_summary(num_trials=50)

Results summary
Results in gru_toxic_dir/untitled_project
Showing 50 best trials
Objective(name="val_auc", direction="max")

Trial 07 summary
Hyperparameters:
activation: elu
optimizer: adam
layers_gru_num: 2
neurons_num: 12
embedding_num: 116
embeddings_regularizer: 0.0001
kernel_regularizer: 0.001
Score: 0.9662976861000061

Trial 28 summary
Hyperparameters:
activation: elu
optimizer: adam
layers_gru_num: 0
neurons_num: 36
embedding_num: 84
embeddings_regularizer: 0.0001
kernel_regularizer: 0.01
Score: 0.9596219062805176

Trial 31 summary
Hyperparameters:
activation: elu
optimizer: adam
layers_gru_num: 0
neurons_num: 36
embedding_num: 84
embeddings_regularizer: 0.0001
kernel_regularizer: 0.01
Score: 0.9593374729156494

Trial 41 summary
Hyperparameters:
activation: elu
optimizer: adam
layers_gru_num: 1
neurons_num: 36
embedding_num: 84
embeddings_regularizer: 0.0001
kernel_regularizer: 0.01
Score: 0.9585903882980347

Trial 26 summary
Hyperparameters:
activation: elu
optimizer: adam
lay

# Загрузка тестовых данных

In [17]:
test_comments = pd.read_csv('drive/MyDrive/toxicCommentsDataSet/toxicCommentsTest.csv')

In [18]:
test_labels = pd.read_csv('drive/MyDrive/toxicCommentsDataSet/toxicCommentsTestLabels.csv')

In [19]:
test_full = pd.merge(test_comments, test_labels, on='id')

In [20]:
test = test_full[test_full['toxic']!=-1]

# Токенизация

In [21]:
test_sequences = tokenizer.texts_to_sequences(test['comment_text'])

In [22]:
y_test = test[['toxic',	'severe_toxic',	'obscene', 'threat',	'insult',	'identity_hate']]

In [23]:
x_test = pad_sequences(test_sequences, maxlen=max_comment_len)

# Проверка лучшей модели

In [24]:
model = tuner.get_best_models(num_models=1)

In [25]:
model[0].evaluate(x_test, y_test)

2000/2000 [==============================] - 63s 31ms/step - loss: 0.0992 - auc: 0.9600 - accuracy: 0.9968


[0.0992300882935524, 0.9599576592445374, 0.9968426823616028]

# Сохранение модели

In [26]:
model[0].save('drive/MyDrive/toxicCommentsDataSet/toxicCommentGRUModel.h5')